### Python Script that creates trial templeates

In [104]:
# Load libraries
import pandas as pd
import os
from configparser import ConfigParser
import numpy as np
import math
import configparser

### Functions

In [105]:
def round_up(n, decimals=0):
    multiplier = 10 ** decimals
    return math.ceil(n * multiplier) / multiplier

### Get config file

In [106]:
#Read config.ini file
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

### Read the base templates

In [107]:
# Roots anatomy templates are in the Folder Fwd__Random_Roots_Anatom
folder = config.get('TEMPLATE', 'folder')
nameTemplateFile = config.get('TEMPLATE', 'nameTemplateFile')
fileNameRead = folder + nameTemplateFile

In [108]:
os.getcwd()

'/mnt/c/Users/luizv/OneDrive_Purdue/OneDrive - purdue.edu/CIMMYT-Purdue RS&CropModelling/Data workflow/fieldBook_template'

In [109]:
# Read the base template
baseTemplate = pd.read_csv(fileNameRead)

# transform column names into lowcase to make them case insensitive
baseTemplate.columns = baseTemplate.columns.str.lower() 
baseTemplate.columns

Index(['plot', 'genotype', 'random_field_no', 'environment'], dtype='object')

### Create the new file

In [110]:
# get the rows to select from template 
template_rows = config.get('TEMPLATE', 'columns')
template_rows_lower = [s.lower().replace(' ', '') for s in template_rows.split(',')]
template_rows_lower

['plot', 'genotype', 'random_field_no', 'environment']

In [111]:
# make the subset from the template file
baseTemplate_subset = baseTemplate[template_rows_lower]

In [112]:
# add the new colmns to the template
new_columns = config.options('NEW_COLUMNS')
number = len(template_rows_lower) # for the dataframe positionof new columns
for new_column in new_columns:
    
    if new_column == 'sampling_identifier':
        # add the text 'sampling' fo the new column if it is 'season_Of_sampling'
        baseTemplate_subset.insert(number, new_column, 'sampling-' + config.get('NEW_COLUMNS', new_column))
    
    else:
        baseTemplate_subset.insert(number, new_column, config.get('NEW_COLUMNS', new_column))
    number += 1

In [113]:
# create the rows for the number of measurements that are going to be taken from each row
timesRepeatRow = len(config.get('NEW_FILE_CONFIG', 'Sample_ID').split(','))
baseTemplate_subset_rep = baseTemplate_subset.loc[baseTemplate_subset.index.repeat(timesRepeatRow)]

In [114]:
baseTemplate_subset_rep

,plot,genotype,random_field_no,environment,experiment,season,measurment,sampling_identifier
0,1,PI594301,3,M2,PPAC-Soybean,y22,Canopy-Temperature,sampling-2
1,2,LD-07-3395bf,23,M2,PPAC-Soybean,y22,Canopy-Temperature,sampling-2
2,3,SA1730464,18,M2,PPAC-Soybean,y22,Canopy-Temperature,sampling-2
3,4,PI154189,14,M2,PPAC-Soybean,y22,Canopy-Temperature,sampling-2
4,5,PI594451,2,M2,PPAC-Soybean,y22,Canopy-Temperature,sampling-2
...,...,...,...,...,...,...,...,...
145,146,PI567780B,12,I1,PPAC-Soybean,y22,Canopy-Temperature,sampling-2
146,147,SA1730464,18,I1,PPAC-Soybean,y22,Canopy-Temperature,sampling-2
147,148,LD-07-3395bf,131,I1,PPAC-Soybean,y22,Canopy-Temperature,sampling-2
148,149,PI39528B,139,I1,PPAC-Soybean,y22,Canopy-Temperature,sampling-2


In [115]:
# get the rows to select from template 
plots_file = config.get('NEW_FILE_CONFIG', 'Sample_ID')
plots = [s.lower().replace(' ', '') for s in plots_file.split(',')]

#columns_row = int(config.get('NEW_FILE_CONFIG', 'columnsPerRow'))
baseTemplate_subset_rep['sample_name'] = plots * int(len(baseTemplate_subset_rep)/len(plots))

# move the'Sample_ID' column to the fist position of the data frame
col = baseTemplate_subset_rep.pop('sample_name')
baseTemplate_subset_rep.insert(0, col.name, col)

In [116]:
# create the plot ID base on the value of other columns
a = baseTemplate_subset_rep['plot'].astype(str)
b = baseTemplate_subset_rep['sample_name']
c = baseTemplate_subset_rep['experiment']
d = baseTemplate_subset_rep['environment']
e = baseTemplate_subset_rep['season']
f = baseTemplate_subset_rep['measurment']
g = baseTemplate_subset_rep['sampling_identifier']
baseTemplate_subset_rep["id_sample"] = a + '_' + b + '_' + c + '_' + d + '_' + e + '_' + f + '_' + g

# move the'id_plot' column to the fist position of the data frame
col = baseTemplate_subset_rep.pop('id_sample')
baseTemplate_subset_rep.insert(0, col.name, col)

In [117]:
baseTemplate_subset_rep

,id_sample,sample_name,plot,genotype,random_field_no,environment,experiment,season,measurment,sampling_identifier
0,1_a_PPAC-Soybean_M2_y22_Canopy-Temperature_sam...,a,1,PI594301,3,M2,PPAC-Soybean,y22,Canopy-Temperature,sampling-2
1,2_a_PPAC-Soybean_M2_y22_Canopy-Temperature_sam...,a,2,LD-07-3395bf,23,M2,PPAC-Soybean,y22,Canopy-Temperature,sampling-2
2,3_a_PPAC-Soybean_M2_y22_Canopy-Temperature_sam...,a,3,SA1730464,18,M2,PPAC-Soybean,y22,Canopy-Temperature,sampling-2
3,4_a_PPAC-Soybean_M2_y22_Canopy-Temperature_sam...,a,4,PI154189,14,M2,PPAC-Soybean,y22,Canopy-Temperature,sampling-2
4,5_a_PPAC-Soybean_M2_y22_Canopy-Temperature_sam...,a,5,PI594451,2,M2,PPAC-Soybean,y22,Canopy-Temperature,sampling-2
...,...,...,...,...,...,...,...,...,...,...
145,146_a_PPAC-Soybean_I1_y22_Canopy-Temperature_s...,a,146,PI567780B,12,I1,PPAC-Soybean,y22,Canopy-Temperature,sampling-2
146,147_a_PPAC-Soybean_I1_y22_Canopy-Temperature_s...,a,147,SA1730464,18,I1,PPAC-Soybean,y22,Canopy-Temperature,sampling-2
147,148_a_PPAC-Soybean_I1_y22_Canopy-Temperature_s...,a,148,LD-07-3395bf,131,I1,PPAC-Soybean,y22,Canopy-Temperature,sampling-2
148,149_a_PPAC-Soybean_I1_y22_Canopy-Temperature_s...,a,149,PI39528B,139,I1,PPAC-Soybean,y22,Canopy-Temperature,sampling-2


In [118]:
# reset dataframe index
baseTemplate_subset_rep = baseTemplate_subset_rep.reset_index(drop=True)
measurment = config.get('NEW_COLUMNS', 'measurment')

#extract elements from config fiule for new file name
measurment = config.get('NEW_COLUMNS', 'measurment')
sampling_identifier = config.get('NEW_COLUMNS', 'sampling_identifier')
experiment = config.get('NEW_COLUMNS', 'experiment')
environment = config.get('NEW_COLUMNS', 'environment')
season = config.get('NEW_COLUMNS', 'season')

# create the name of the file
nameNewFile = folder + measurment + '_' + sampling_identifier + '_' + experiment + '_' + season + '.csv'
#nameNewFile = folder + measurment + '_' + sampling_identifier + '_' + experiment + '_' + environment + '_' + season + '.csv'
nameNewFile
# export file into csv
baseTemplate_subset_rep.to_csv(nameNewFile, index=False)